### QA
#### - Hacer análisis descriptivo y control de datos con Python.
#### - Explicar hallazgos y posibles problemas. Proponer posibles soluciones de forma escrita y/o profundizando en el video.

In [1]:
import boto3
import pandas as pd
from prettytable import PrettyTable
import csv
from botocore.exceptions import NoCredentialsError

def detect_delimiter(file_name):
    with open(file_name, 'r', newline='', encoding='utf-8') as csvfile:
        try:
            dialect = csv.Sniffer().sniff(csvfile.readline(), delimiters=';,\t|')
            delimiter = dialect.delimiter
            print(f"Delimiter for {file_name}: '{delimiter}'")
            return dialect.delimiter
        except csv.Error:
            return ','

def print_table(df, num_rows=5):
    table = PrettyTable()
    table.field_names = df.columns.tolist()
    for index, row in df.head(num_rows).iterrows():
        table.add_row(row.values)
    print(table)

def descriptive_analysis(df):
    """
    Realiza un análisis descriptivo del DataFrame.
    """
    print("\nDescripción estadística:")
    print(df.describe(include='all'))

    print("\nInformación del DataFrame:")
    print(df.info())

    print("\nConteo de valores nulos:")
    print(df.isnull().sum())

def data_quality_checks(df):
    """
    Realiza verificaciones de calidad de los datos.
    """
    print("\nVerificaciones de calidad de datos:")
    # Duplicados
    duplicates = df.duplicated().sum()
    print(f"\nNúmero de filas duplicadas: {duplicates}")

    # Valores únicos por columna
    for column in df.columns:
        unique_count = df[column].nunique()
        print(f"{column} - Valores únicos: {unique_count}")

def analyze_dataframes(dataframes):
    for name, df in dataframes.items():
        print(f"\nAnálisis de {name}:")
        print_table(df)
        descriptive_analysis(df)
        data_quality_checks(df)

dataframes = {}
file_paths = ['disney_plus_titles.csv', 'netflix_titles.csv']
for file_path in file_paths:
    local_file_name = file_path.split('/')[-1]
    delimiter = detect_delimiter(local_file_name)
    df = pd.read_csv(local_file_name, delimiter=delimiter, quotechar='"', escapechar='\\', on_bad_lines='skip')
    dataframes[local_file_name] = df

analyze_dataframes(dataframes)

Delimiter for disney_plus_titles.csv: ','
Delimiter for netflix_titles.csv: ';'

Análisis de disney_plus_titles.csv:
+---------+---------+--------------------------------------------------+-----------------------------------+----------------------------------------------------------------------------------------------+---------------+-------------------+--------------+--------+----------+-------------------------------+---------------------------------------------------------------------------------------------------+
| show_id |   type  |                      title                       |              director             |                                             cast                                             |    country    |     date_added    | release_year | rating | duration |           listed_in           |                                            description                                            |
+---------+---------+------------------------------------------------

## Explicación de Hallazgos y Posibles Problemas
Al implementar y ejecutar el código proporcionado para analizar los archivos CSV de Netflix y Disney+, nos encontramos con problemas comunes en el análisis de datos.

### 1. Diferentes Delimitadores en Archivos CSV:

Los archivos pueden tener diferentes delimitadores, como comas, puntos y coma, o tabulaciones.
Pandas por defecto espera comas como delimitadores, lo que puede llevar a errores de lectura si el archivo usa otro delimitador.
El código implementa una función detect_delimiter que usa csv.Sniffer para inferir automáticamente el delimitador más probable. Esto permite cargar correctamente los DataFrames independientemente del delimitador utilizado.

### 2. Presencia de Valores Nulos:

Los DataFrames pueden tener una cantidad significativa de valores nulos.
Los valores nulos pueden afectar el análisis estadístico y las operaciones de modelado si no se manejan adecuadamente.
El análisis descriptivo incluye un conteo de valores nulos por columna, lo que permite identificar las columnas que necesitan atención para limpieza de datos o imputación de valores.

En ambos casos el campo "director" es el que mayor cantidad de valores nulos tiene:
- En el CSV de Disney+: 609 valores únicos, 473 valores nulos
- En el CSV de Netflix: 4528 valores únicos, 2636 valores nulos

Otros dos campos que tienen bastantes valores nulos en ambos casos son: "cast" y "country".

### 3. Datos Duplicados:

Puede haber filas duplicadas en los datasets.
Los duplicados pueden distorsionar el análisis y llevar a conclusiones erróneas.
Se realiza un conteo de duplicados para cada DataFrame. La limpieza de estos duplicados, si es necesario, se puede realizar mediante df.drop_duplicates().

En ambos casos el número de filas duplicadas es: 0

### 4. Inconsistencias en los Datos:

Las columnas pueden tener tipos de datos inconsistentes o inapropiados, como fechas almacenadas como strings.
Esto puede complicar las operaciones de partición, filtrado, ordenación y cálculos.
La función descriptive_analysis incluye un chequeo de df.info(), que muestra los tipos de datos de cada columna, permitiendo la detección de tipos incorrectos.

- En ambos casos el campo "date_added", el cual usaremos para particionar las tablas, viene en el siguiente formato (STRING): "January 1, 2020".
Como solución vamos a cambiar el formato del campo "date_added" para que sea tipo DATE.

#### df['date_added'] = pd.to_datetime(df['date_added'], format='%B %d, %Y', errors='coerce')
#### df['date_added'] = pd.to_datetime(df['date_added']).dt.date

- Por otro lado, el campo "release_year", el formato es "2018". En el caso de Disney+ el tipo de dato lo detecta como INT64, y en el caso de Netflix como STRING.
En este caso como solución vamos a homogeneizar los datos transformando el campo para que sea INT64 en todos los casos. Luego vamos a clusterizar las tablas basadas en este campo.

#### df['release_year'] = pd.to_numeric(df['release_year'], downcast='integer', errors='coerce')

- Otra inconsistencia que encontramos son unas comillas que en el campo cast generan algunos errores a la hora de cargar los datos en la base de datos.
Esto lo solucionamos trimeando las comillas de ese campo.

#### df['cast'] = df['cast'].str.strip('"')

- Por último, dado que el "show_id" se duplica al hacer el union de ambas tablas, se crea un id único concatenando el nombre del servicio de streaming con el "show_id".

#### df['unique_id'] = service_name + '_' + df['show_id'].astype(str)